In [16]:
# imports
# from collections import Counter
# from itertools import islice
import re
import pandas as pd
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import math
import operator
import numpy as np
porter = PorterStemmer()
# #########################
# DATA CLEANING TECHNIQUES
# #########################
# split by sentance
# remove puncuation
# lowercase letters
# remove/convert numbers
# -------------------------

# #########################
# NLP TECHNIQUES
# #########################
# stemming
# -------------------------
# sent1 = "i am sam"
# sent2 = "sam i am"
# sent3 = "i do not like green eggs and ham"
# sent4 = "the quick person did not realize his speed and the quick person bumped "


# # words = re.findall("\w+", sent4)
# # bigramCount = dict(Counter(zip(words, islice(words, 1, None))))

# # for i in bigramCount.keys():
# #     print(str(i) + " count: %d" % bigramCount[i])

In [62]:
stopwords = {"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "that", "thats", "that's", "than", "too", "very", "can", "will", "just", "should", "now"}
gibberish = ['\(CNN\)', 'CNN', 'Getty Images', '\\\\xc2', '\\\\xb0', '\\\\xc3', '\\\\x94', '\\\\x9c', '\\\\x9d', '\\\\x95', '\\\\xb3', '\\\\xe2', '\\\\x80', '\\\\x99', '\\\\xa9', '\\\\xf0', '\\\\x9f', '\\\\x91']
collection = str.maketrans('', '', string.punctuation)

# load in the crawled data and perform some cleaning
news = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\final.csv")
print(news.shape[0])
# news = news[:50]

# for i in range(news.shape[0]):
#     s = news.iloc[i][3][2:]
#     s2 = news.iloc[i][1][2:]
#     news.at[i,'body'] = s
#     news.at[i,'headline'] = s2

print("done")


for i in range(len(gibberish)):
    for j in range(news.shape[0]):
        output = news.iloc[j][4]
        output = re.sub(gibberish[i], '', output)
        output = re.sub('\\\\n', ' ', output)
        output = re.sub('\\\\\\\'', '', output)
        output = re.sub('\'', '', output)
        output = re.sub('-', ' ', output)
        output = re.sub('Read More', '', output)
        news.at[j,'body'] = output

print("############## all clean and ready for cross-val ################")

7009
done
############## all clean and ready for cross-val ################


In [28]:
def cleantokens(news):
    dterms = {}
    doc_labels = {}
    for j in range(news.shape[0]):
        tokens = word_tokenize(news.iloc[j][4])
        tokens = [t.lower() for t in tokens]
        depunc = [t.translate(collection) for t in tokens]
        words = [w for w in depunc if not w  == '']
        cleaner = [w for w in words if not w in stopwords]
        stemmed = [porter.stem(word) for word in cleaner]
        dterms[str(j)+news.iloc[j][2]] = stemmed
        doc_labels[str(j)+news.iloc[j][2]] = news.iloc[j][6]
    #     dterms[j] = cleaner
    return dterms, doc_labels

In [19]:
# doc_labels = {}
# dterms = {}
# term_list = []
# doc_index = {}
# term_index = {}
# words_in_doc = {}

# valid_doc_labels = {}
# valid_dterms = {}
# valid_term_list = []
# valid_doc_index = {}
# valid_term_index = {}
# valid_words_in_doc = {}

#function that creates termlist and doc_idx as we count each doc_length
def populate_data_structs(dterms, words_in_doc, term_list, doc_index):
    for doc in dterms.keys():
        count = 0
        words_in_doc[doc] = set({})
        for word in dterms[doc]:
            count += 1
            term_list.append((word, doc))
            words_in_doc[doc].add(word)
        doc_index[doc] = count


def make_term_index(term_list, term_index):
    def update_term(item):
        i = 0
        for listitem in term_index[item[0]]:
            #if word_docName == encounterd_fileID
            if (item[1] == listitem[0]):
                listitem[1] += 1
                return
            else:
                i += 1
        term_index[item[0]].append( [item[1] , 1 ] )
    
    for item in term_list:
        if item[0] not in term_index:
            # add term
            term_index[item[0]] = [[item[1], 1]]
        else:
            # update term
            update_term(item)

In [20]:
def cosine_score(query_words, doc_index, term_index):
    def get_posting_list(term):
        return term_index[term] if term in term_index else []
    
    def get_q_weight(term):
        tf = 0
        for w in query_words:
            if w == term:
                tf += 1
        tf = tf / len(query_words)
        if term in term_index:
            idf = 1 + math.log(len(doc_index)/len(term_index[term]))
        else:
            idf = 1
        return tf * idf
    
    def get_d_weight(term, pair):
        if term in term_index:
            tf = pair[1]/doc_index[pair[0]] #num_occur/doc_size
            idf = 1 + math.log(len(doc_index)/len(term_index[term]))
            tf_idf = tf * idf
        else:
            tf_idf = 0
        return tf_idf

    scores = {}
    for doc in doc_index.keys():
        scores[doc] = 0
    
    for term in query_words:
        for pair in get_posting_list(term):
            scores[pair[0]] += get_q_weight(term) * get_d_weight(term, pair)
    qlength = len(query_words)
    for doc in doc_index.keys():
        scores[doc] = scores[doc] / (doc_index[doc]*qlength)
        
    sorted_scores = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_scores[:3]

In [ ]:
def getpotentials(qu, s, t, e, p, o, maxval):
    pot = []
    if qu == s and qu == maxval:
        pot = ['quarantine', 'social distancing']
    elif qu == t and qu == maxval:
        pot = ['quarantine', 'treatment']
    elif qu == p and qu == maxval:
        pot = ['quarantine', 'pandemic']
    elif qu == e and qu == maxval:
        pot = ['quarantine', 'economic effects']
    elif qu == o and qu == maxval:
        pot = ['quarantine', 'other']
    elif s == t and s == maxval:
        pot = ['social distancing', 'treatment']
    elif s == p and s == maxval:
        pot = ['social distancing', 'pandemic']
    elif s == e and s == maxval:
        pot = ['social distancing', 'economic effects']
    elif s == o and s == maxval:
        pot = ['social distancing', 'other']
    elif t == p and t == maxval:
        pot = ['treatment', 'pandemic']
    elif t == e and t == maxval:
        pot = ['treatment', 'economic effects']
    elif t == o and t == maxval:
        pot = ['treatment', 'other']
    elif p == e and p == maxval:
        pot = ['pandemic', 'economic effects']
    elif p == o and p == maxval:
        pot = ['pandemic', 'other']
    else:
        pot = ['economic effects', 'other']
    return pot


three_nearest_neighbors = {}
k = 3

quarentine = 'quarantine'
socialdist = 'social distancing'
treat = 'treatment'
econeff = 'economic effects'
pandemic = 'pandemic'
other = 'other'
splits = 24
pointspersplit = 5
errors = []
for i in range(splits):
    end = pointspersplit*(i+1)
    start = end - pointspersplit
    valid = news.iloc[start:end][:]
    train = pd.concat([news.iloc[:start][:], news.iloc[end:][:]])
#     make all of these:
    dterms, doc_labels = cleantokens(train)
    term_list = []
    doc_index = {}
    words_in_doc = {}
    populate_data_structs(dterms, words_in_doc, term_list, doc_index)
    term_index = {}
    make_term_index(term_list, term_index)
    valid_dterms, valid_doc_labels = cleantokens(valid)
    error = 0
    
    for q in valid_dterms.keys():
        qu = s = t = e = p = o = 0
        labels = []
        scores = cosine_score(valid_dterms[q], doc_index, term_index)
        for j in range(len(scores)):
            label = doc_labels[scores[j][0]]
            labels.append(label) # append each label from closest to farthest
            if label == 'quarantine':
                qu += 1
            elif label == 'social distancing':
                s += 1
            elif label == 'treatment':
                t += 1
            elif label == 'economic effects':
                e += 1
            elif label == 'pandemic':
                p += 1
            else:
                o += 1
        top2 = sorted([qu, s, t, e, p, o], reverse = True)[:2]
        max_label = ''
        if top2[0] == top2[1]:
            # pick the closer one
            max_label = ''
            potentials = getpotentials(qu, s, t, e, p, o, top2[0])
            for l in labels:
                if l == potentials[0] or l == potentials[1]:
                    if l == potentials[0]:
                        max_label = potentials[0]
                    else:
                        max_label = potentials[1]
                    break
        else:
#             max_count = np.max([qu, s, t, e, p, o])
            max_count = top2[0]
            if max_count == qu:
                max_label = 'quarantine'
            elif max_count == s:
                max_label = 'social distancing'
            elif max_count == t:
                max_label = 'treatment'
            elif max_count == e:
                max_label = 'economic effects'
            elif max_count == p:
                max_label = 'pandemic'
            else:
                max_label = 'other'
        if not max_label == valid_doc_labels[q]:
            error += 1
        three_nearest_neighbors[q] = [scores, max_label]
    errors.append(error)
    
print('Average error per validation split: %f%' % (np.mean(errors)/pointspersplit))

In [63]:
keywords = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\labelandKeywords.csv")
print(keywords.shape[0])
display(keywords.head())

5


,label,keywords
0,quarantine,Shelter In Place \tWork From Home \tRemote Edu...
1,social distancing,Flatten The Curve \tFace Masks \tStay Home \t6...
2,treatment,Emerging Data\tClinical Trials\tDetection\tTes...
3,economic effects,Stock Market\tincome\tjob\tlayoff\tSmall Busin...
4,pandemic,Outbreak\t Reopening\tInfluenza\tclosed\tdeath...


In [64]:
label_keywords = {}
for j in range(keywords.shape[0]):
    tokens = word_tokenize(keywords.iloc[j][1])
    tokens = [t.lower() for t in tokens]
    cleaner = [w for w in tokens if not w in stopwords]
    stemmed = [porter.stem(word) for word in cleaner]
    label_keywords[keywords.iloc[j][0]] = stemmed




In [65]:
dterms, doc_labels = cleantokens(news)
# term_list = []
# doc_index = {}
# words_in_doc = {}
# populate_data_structs(dterms, words_in_doc, term_list, doc_index)
# term_index = {}
# make_term_index(term_list, term_index)
article_covid_labels = []

words_in_labels = {}
q_index = {}
qterm_list = []
qterm_index = {}
populate_data_structs(label_keywords, words_in_labels, qterm_list, q_index)
make_term_index(qterm_list, qterm_index)

for doc in dterms.keys():
    scores = cosine_score(dterms[doc], q_index, qterm_index)
    article_covid_labels.append(scores[0][0])

news['covid labels']  = article_covid_labels

news.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\covidLabeld.csv', index = False, header=True)
    
# df = pd.DataFrame.from_dict(three_nearest_neighbors, orient='index', columns=['3-NN', 'Label'])
# df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\ThreeNN.csv', index = True, header=True)
i = 0
# for doc in dterms.keys():
#     print(doc)
#     print(article_covid_labels[i])
#     i += 1


In [11]:
def termweight(term, docno):
    if term in words_in_doc[docno]:
        for postings in term_index[term]:
            if postings[0] == docno:
                return postings[1]/doc_index[postings[0]] * math.log(len(doc_index)/len(term_index[term]), 2)
    return 0


term_doc_matrix = {}

for term in term_index.keys():
    row = []
    for doc in doc_index.keys():
        row.append(termweight(term, doc))
    term_doc_matrix[term] = row


# df = pd.DataFrame.from_dict(term_doc_matrix, orient='index', columns=keys)
# df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFTrainSet.csv', index = True, header=True)

term_list = []
# doc_index = {}
# term_index = {}
words_per_doc = {}
# term_doc_matrix = {}


In [ ]:
# populate_data_structs()
# keys = list(doc_index.keys())
# make_term_index()

In [19]:
# @param q: the column index of the query article, or the column name of the query article, I think the latter would work
def CosineScore(q):
    scores = {}
    query = weightmatrix.loc[:,q].values
    normq = (query@query)
    for doc in doc_index.keys():
        if not doc == q:
            # divide by the values of the weight vector norms
            d = weightmatrix.loc[:,doc].values
            normd = d@d
            scores[doc] = (query@d)/((normq*normd)**0.5)
    sorted_scores = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_scores[:3]


In [24]:
# weightmatrix = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFMatrix.csv")
# weightmatrix = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFTrainSet.csv")
# print(weightmatrix.shape)
# display(weightmatrix.head())


In [44]:
# df = pd.DataFrame.from_dict(term_doc_matrix, orient='index', columns=['3-NN', 'Label'])
# df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFTrainSet.csv', index = True, header=True)


In [2]:
# initialize the set and the dataframe with the first word of the first document
# words_encountered = set(dterms[0][0])
# unigrams = pd.DataFrame({dterms[0][0]: [0]})

# for i in range(news.shape[0]):
#     if i > 0:
#         tdf = pd.DataFrame([[0]*len(unigrams.columns)], columns=unigrams.columns)
#         t = unigrams.append(tdf,ignore_index=True)
#         unigrams = t
#     for word in dterms[i]:
#         if word not in words_encountered:
#             words_encountered.add(word)
#             col_index = len(unigrams.columns)
#             temp = pd.DataFrame({word: [0]*(i+1)})
#             t2 = unigrams.join(temp)
#             unigrams = t2
#             unigrams.at[i, word] = 1
#         else:
#             cols = list(unigrams.columns)
#             unigrams.at[i, word] += 1
# too slow with dataframes ^ 

# for i in range(10):
#     print(dterms[i][:6])

# unigrams.to_csv('unigrams2.csv', encoding='utf-8')

# script for creating term index csv
# for k in dterms.keys():
#     print(k, dterms[k][0])

# a_file = open("TermDocMatrix.csv", "w")

# writer = csv.writer(a_file)
# for key, value in term_index.items():
#     writer.writerow([key, value])

# a_file.close()